In [3]:
#kaggle competitions download -c home-credit-default-risk
%cd ~/.kaggle/competitions/home-credit-default-risk/
!unzip '*.zip'

/Users/muneyoshiinahara/.kaggle/competitions/home-credit-default-risk
Archive:  bureau_balance.csv.zip
  inflating: bureau_balance.csv      

Archive:  application_train.csv.zip
  inflating: application_train.csv   

Archive:  credit_card_balance.csv.zip
  inflating: credit_card_balance.csv  

Archive:  application_test.csv.zip
  inflating: application_test.csv    

Archive:  installments_payments.csv.zip
  inflating: installments_payments.csv  

Archive:  POS_CASH_balance.csv.zip
  inflating: POS_CASH_balance.csv    

Archive:  bureau.csv.zip
  inflating: bureau.csv              

Archive:  previous_application.csv.zip
  inflating: previous_application.csv  

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

9 archives were successfully processed.


In [4]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# automated feature engineering
import featuretools as ft

# Filter out pandas warnings
import warnings 
warnings.filterwarnings('ignore')

In [5]:
app_train = pd.read_csv('./application_train.csv').sort_values('SK_ID_CURR').reset_index().loc[:1000, :].drop(columns = ['index'])
app_test = pd.read_csv('./application_test.csv').sort_values('SK_ID_CURR').reset_index().loc[:1000, :].drop(columns = ['index'])
bureau = pd.read_csv('./bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index().loc[:1000, :].drop(columns = ['index'])
bureau_balance = pd.read_csv('./bureau_balance.csv').sort_values('SK_ID_BUREAU').reset_index().loc[:1000, :].drop(columns = ['index'])
cash = pd.read_csv('./POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(columns = ['index'])
credit = pd.read_csv('./credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(columns = ['index'])
previous = pd.read_csv('./previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(columns = ['index'])
installments = pd.read_csv('./installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(columns = ['index'])

In [6]:
app_train[0:3]

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
app_types = {}
# Iterate through the columns and record the Boolean columns
for col in app_train:
    # If column is a number with only two values, encode it as a Boolean
    if (app_train[col].dtype != 'object') and (len(app_train[col].unique()) <= 2):
        app_types[col] = ft.variable_types.Boolean
print('Number of boolean variables: ', len(app_types))

Number of boolean variables:  33


In [8]:
app_types

{'FLAG_CONT_MOBILE': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_10': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_11': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_12': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_13': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_14': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_15': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_16': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_17': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_18': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_19': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_2': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_20': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_21': featuretools.variable_types.variable.Boolean,
 'FLAG_DOCUMENT_3': featuretools.variable_types.variable.Boolea

In [ ]:
# Record ordinal variables
app_types['REGION_RATING_CLIENT'] = ft.variable_types.Ordinal
app_types['REGION_RATING_CLIENT_W_CITY'] = ft.variable_types.Ordinal
app_test_types = app_types.copy()
del app_test_types['TARGET']